In [1]:
using Laplacians

objc[48083]: Class TKApplication is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[48083]: Class TKMenu is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[48083]: Class TKContentView is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[48083]: Class TKWindow is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.


In [2]:
set_bigfloat_precision(1024)

1024

In [66]:
u = Array{Int64,1}(0)
v = Array{Int64,1}(0)
w = Array{Float64,1}(0)

n = 200
for i in 1:(n-1)
    val = rand() * 10^9;
    
    push!(u, i)
    push!(v, i+1)
    push!(w, val)
    
    push!(u, i+1)
    push!(v, i)
    push!(w, val)
end

utree = Array{Int64,1}(0)
vtree = Array{Int64,1}(0)
wtree = Array{Float64,1}(0)

extra = n
for i in 1:(n-1)
    if i % 2 == 1
        extra = extra + 1
        
        push!(u, i)
        push!(v, extra)
        push!(w, 1)
        
        push!(u, extra)
        push!(v, i)
        push!(w, 1)
        
        push!(u, i + 1)
        push!(v, extra)
        push!(w, 1)
        
        push!(u, extra)
        push!(v, i + 1)
        push!(w, 1)
        
        
        
        
        
        push!(utree, i)
        push!(vtree, extra)
        push!(wtree, 1)
        
        push!(utree, extra)
        push!(vtree, i)
        push!(wtree, 1)
        
        push!(utree, i + 1)
        push!(vtree, extra)
        push!(wtree, 1)
        
        push!(utree, extra)
        push!(vtree, i + 1)
        push!(wtree, 1)
    end
end

a = sparse(u,v,w); la = lap(a);
tree = lap(sparse(utree,vtree,wtree));
n = a.n

300

In [67]:
# n = 1250;
# a = chimera(n,44); la = lap(a);
x = rand(n);
b = la * x; b = b - mean(b);

In [68]:
srand(1)
# tree = lap(akpw(a)); 
cf = cholfact(tree,shift=1e-7);
p = cf[:p]
invp = invperm(p)
L = sparse(cf[:L])
U = L';

In [69]:
F = function(b)
    return cf \ (b - mean(b))
end

(anonymous function)

In [70]:
@time f = pcg(la, b, F, tol=1e-7, verbose=true);

PCG stopped after: 2101 iterations with relative error 8.847073924930903e-8.
  0.099173 seconds (51.93 k allocations: 54.770 MB, 17.88% gc time)


In [71]:
function to1024{Tv,Ti}(mat::SparseMatrixCSC{Tv,Ti})
    u,v,w = findnz(mat)
    w1024 = Array{BigFloat,1}(0)
    for val in w
        push!(w1024, BigFloat(val))
    end
    return sparse(u,v,w1024)
end

function to1024{Tv}(v::Array{Tv,1})
    res = Array{BigFloat,1}(0)
    for val in v
        push!(res, BigFloat(val))
    end
    return res
end

to1024 (generic function with 2 methods)

In [72]:
la1024 = to1024(la)
U1024 = UpperTriangular(to1024(U))
b1024 = to1024(b);

In [73]:
F1024 = function(b::Array{BigFloat,1})
    res = (copy(b) - mean(b))[p]

    res = U1024' \ res
    res = U1024 \ res

    return res[invp]
end

(anonymous function)

In [74]:
@time f = pcg(la1024, b1024, F1024, tol=1e-3, verbose=true);

PCG stopped after: 371 iterations with relative error 9.218965260662450393247146629134324175787506136521877774486824038320657017029347043142194791503024840659829206314570551513338007909889076358317005934377191068548975882311329024460166770105719385594613151004744684761301024876003008621300428915566358188356552402594873659272756447198457175441897061874764473362993e-04.
 29.111861 seconds (158.48 M allocations: 9.851 GB, 42.44% gc time)
